## Detecting Sequences

The engine can detect sequential patterns of assertions. As these rules trigger, they get saved into memory as "arcs", or partially completed rules. Those arcs wait until they are completed before triggering their assertions.

This can be useful in natural language and other real-world situations where it's important to assign meaning to a sequence of assertions that occur one after the other. These sequences can in turn form parts of larger sequences and structures, and you can carry information from the lower sequences and structures into the high-order sequences and structures to build up a structure of meaning.

To begin, rev up the engine:

In [1]:
import os, sys
sys.path.insert(1, os.path.abspath('..\\..'))
from thoughts.rules_engine import RulesEngine
from pprint import pprint

# start a new engine
engine = RulesEngine()

## Asserting a Partial Sequence

Let's try a simple example in matching the sequence of letters A, B, and C.

At first let's just assert only A and B and inspect the result.

In [2]:
rules = { "#when": ["A", "B", "C"], "#then": "FOUND" }
engine.add_rule(rules)

result = engine.process(["A", "B"], extract_conclusions=True)
pprint(result)

['A', 'B']


Only the incoming assertions were returned, and the rule did not trigger. This makes sense, as the engine is waiting on the final part of the sequence, "C".

## Asserting the Full Sequence

Now let's try asserting the full sequence:

In [3]:
result = engine.process(["A", "B", "C"], extract_conclusions=True)
pprint(result)

['A', 'B', 'FOUND']


Here all three parts, or *constituents*, of the sequence were found. This triggered the final "FOUND" fact.

But why wasn't 'C' returned in the final result? This is an effect of using the extract_conclusions parameter, which only returns the leaf-node (bottom level) facts that were concluded.

Let's take a look at the full tree to make this more apparent:

In [4]:
result = engine.process(["A", "B", "C"], extract_conclusions=False)
pprint(result)

[{'#assert': 'A', '#conclusions': [], '#seq-end': 1, '#seq-start': 0},
 {'#assert': 'B', '#conclusions': [], '#seq-end': 2, '#seq-start': 1},
 {'#assert': 'C',
  '#conclusions': [{'#assert': 'FOUND',
                    '#conclusions': [],
                    '#seq': [{'#assert': 'A', '#seq-end': 1, '#seq-start': 0},
                             {'#assert': 'B', '#seq-end': 2, '#seq-start': 1},
                             {'#assert': 'C', '#seq-end': 3, '#seq-start': 2}],
                    '#seq-end': 3,
                    '#seq-start': 0}],
  '#seq-end': 3,
  '#seq-start': 2}]


Above, 'A' and 'B' have no sub-conclusions, so they are returned. 'C' has the sub-conclusion of 'FOUND' and so it is *not* returned. 'FOUND' has no sub-conclusions, so it is returned.

## Inspecting Arcs

Now let's run the rule again but this time we'll inspect the partial rules, or arcs, which are waiting on the next constituent in the sequence.

In [5]:
result = engine.process(["A", "B"], extract_conclusions=True)

pprint(engine.context.arcs, sort_dicts=False)

[{'#when': [{'#assert': 'A', '#seq-start': 0, '#seq-end': 1}, 'B', 'C'],
  '#then': 'FOUND',
  '#seq-idx': 1,
  '#seq-start': 0,
  '#seq-end': 1,
  '#unification': {'?#when': {'#assert': 'A', '#seq-start': 0, '#seq-end': 1}},
  '#is-arc': True},
 {'#when': [{'#assert': 'A', '#seq-start': 0, '#seq-end': 1},
            {'#assert': 'B', '#seq-start': 1, '#seq-end': 2},
            'C'],
  '#then': 'FOUND',
  '#seq-idx': 2,
  '#seq-start': 0,
  '#seq-end': 2,
  '#unification': {'?#when': {'#assert': 'B', '#seq-start': 1, '#seq-end': 2}},
  '#is-arc': True}]


Above, there are two entries in the partial rules. Notice the system-generated #seq-idx, #seq-start, and #seq-end attributes.

The first entry has detected the 'A' constituent, "covering" positions 0 and 1. The rule is "sitting" at index position 1, meaning that it's waiting for the constiuent at that position, which in this example is 'B'. 

The second entry has detected the 'B' consituent, and has extended the original arc by covering the position from 0 to 2, which includes both the 'A' and 'B' positions. the index is sitting at position 2, meaning that it's waiting for the 'C' constituent.

*In general, you can think of the arcs as both "covering" positions and "sitting" at the last position, waiting on the next constituent to arrive.*

Also note that *both* arcs are kept in the partial rules. The engine keeps all partial matches to rules, even after extending an arc. This is how it keeps a list of all possibilities, in case another constituent arrives which could also match the rule.

Finally, the arcs are cleared by the engine automatically each time the process function runs.

## Constituents Must Follow Immediately After Each Other

By default, the engine will only detect sequences where the next token constituent is immediately following the previous constituent. 

It assigns a #seq-start and #seq-end token to facts as they are asserted in the process function. Then it compares to make sure the fact immediately follows the previous constituent in the arc, based on the #seq-end and #seq-start positions. 

Compare:

In [6]:
result = engine.process(["A", "B", "C"], extract_conclusions=True)
pprint(result)

['A', 'B', 'FOUND']


To:

In [7]:
result = engine.process(["A", "B", "D", "C"], extract_conclusions=True)
pprint(result)

['A', 'B', 'D', 'C']


Because "C" was not immediately following "A" and "B", the sequence is not detected.

We'll see below how to chance this default behavior to allow for "junk" in the sequence.

## Overlapping Sequences are Detected

Sequences can overlap each other. This works differently than Sets, which by default prevent the same constituent from being matched multiple times in the same rule. See the tutorial on Sets for more details.

The rule below looks for any ABA sequences.

In [8]:
rule = { "#when": ["A", "B", "A"], "#then": "FOUND" }
engine.clear_rules()
engine.add_rule(rule)

result = engine.process(["A", "B", "A", "B", "A"], extract_conclusions=True)
pprint(result)

['A', 'B', 'FOUND', 'B', 'FOUND']


The ABA sequence appears twice in the assertion ABABA, so the rule triggers twice. The middle "A" is part of both sequences.

## Allowing Disconnected Sequences (aka Junk in Sequences)

By default, sequence matching requires that tokens immediately follow each other, with no "junk" tokens in between. You can change this behavior to allow junk by using the #allow-junk attribute.

In [10]:
rule = { "#when": ["A", "B", "C"], 
         "#seq-type": "allow-junk", 
         "#then": "FOUND" }

engine.clear_rules()
engine.add_rule(rule)
engine.process(["A", "B", "D", "E", "F", "C"])

['A', 'B', 'D', 'E', 'F', 'FOUND']

Allowing junk in sequences may at first appear to "overmatch" and generate more sequences than you expect. 

Consider the following:

In [11]:
rule = { "#when": ["A", "B", "C"], 
         "#seq-type": "allow-junk", 
         "#then": "FOUND" }

engine.clear_rules()
engine.add_rule(rule)
engine.process(["A", "B", "X", "A", "B", "C"])

['A', 'B', 'X', 'A', 'B', 'FOUND', 'FOUND', 'FOUND']

At first glance it looks like there are two valid A B C sequences in A B X A B C - the sequence starting in the with the intitial A B and completed by the final C, and the A B C sequence at the end.

However, if you look closely, there are actually three valid sequences of A B C:

+ A B . . . C
+ A . . . B C
+ . . . A B C

For this reason, use the #allow-junk rule carefully to make sure you are getting the results you want.

## Summary

Sequence-based rules are a powerful way for the engine to wait on information to arrive before making conclusions (triggering rules).

Next we'll see a way to detect Sets, where the order the information arrives does not matter.